In [22]:
import numpy as np
import scipy.io as sio
import os
import glob
from scipy import sparse
import random as rd
import os
# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
# import theano
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Convolution3D
from keras.layers import MaxPooling3D
from keras.layers import ZeroPadding3D
from keras.layers import Dropout
from keras.layers import Flatten
import keras
import pandas as pd
import csv
from random import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
seed = 7
np.random.seed(seed)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import re

In [6]:
def readFullSpecFeature(file_name):
    content = sio.loadmat(file_name)
#     print content
    return content['fullspecData']

def loadFullSpecFeature(species, rootPath = '../data/specData/', timeNum = 2, seedsNum = 48):
    data = list()
    for time in range(0, timeNum):
        for seed in range(0,seedsNum):
            file_name = rootPath + species + '-0' + str(time +1) + '/' + str(seed + 1)+"_"+"spec.mat"
#             print file_name
            data.append(readFullSpecFeature(file_name))
    return np.asarray(data)

In [7]:
a = loadFullSpecFeature('HoangLong')
print a.shape

(96, 48, 48, 256)


In [41]:
def C3D(summary = False):
    """ Return the Keras model of the network
    """
    model = Sequential()
    # 1st layer group
    model.add(Convolution3D(64, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv1',
                            subsample=(1, 1, 1), 
                            input_shape=(48, 48, 24, 3)))
#     print model.output_shape
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           border_mode='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(128, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv2',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv3a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv3b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool3'))
    # 4th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv4a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv4b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool4'))
    # 5th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv5a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv5b',
                            subsample=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(1000, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(6, activation='relu', name='fc7'))
#     model.add(Dropout(.5))
#     model.add(Dense(6, activation='softmax', name='fc8'))
    if summary:
        print(model.summary())
    return model

In [43]:
 C3D(summary = True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 48, 48, 24, 64)    5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 48, 24, 12, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 48, 24, 12, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 24, 12, 6, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 24, 12, 6, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 24, 12, 6, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 12, 6, 3, 256)     0         
__________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), name="conv1", activation="relu", input_shape=(48, 48, 2..., padding="same", strides=(1, 1, 1))`
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(padding="valid", strides=(1, 2, 2), name="pool1", pool_size=(1, 2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), padding="same", strides=(1, 1, 1), activation="relu", name="conv2")`
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(padding="valid", strides=(2, 2, 2), name="pool2", pool_size=(2, 2, 2))`
/usr/local/lib/python2.7/dis